In [3]:
# load necessary libraries
import pandas
import statsmodels.api as stats
import sympy
import scipy
import numpy
import sklearn.ensemble as ensemble
import math

print()
print("=" * 50)
print("=" * 50)
print("ML-Assignment 4-Question 1")
print("=" * 50)
print("=" * 50)
# Please answer the following questions based on your model.


ML-Assignment 4-Question 1


In [6]:
# create interaction parameters for model
def create_interaction(in_df1, in_df2):
    name1 = in_df1.columns
    name2 = in_df2.columns
    out_df = pandas.DataFrame()
    for col1 in name1:
        for col2 in name2:
            outName = col1 + " * " + col2
            out_df[outName] = in_df1[col1] * in_df2[col2]
    return (out_df)

In [7]:
# function that crate mnlogit model and return llk,df,full params
def build_mnlogit(full_x, y, debug='N'):
    no_full_param = full_x.shape[1]

    y_category = y.cat.categories
    no_y_cat = len(y_category)

    reduced_form, inds = sympy.Matrix(full_x.values).rref()

    if (debug == 'Y'):
        print('Column Numbers of the Non-redundant Columns:')
        print(inds)

    x = full_x.iloc[:, list(inds)]

    this_df = len(inds) * (no_y_cat - 1)

    logit = stats.MNLogit(y, x)
    this_fit = logit.fit(method='newton', full_output=True, maxiter=100, tol=1e-8)
    this_parameter = this_fit.params
    this_llk = logit.loglike(this_parameter.values)

    if (debug == 'Y'):
        print(this_fit.summary())
        print("Model Parameter Estimates:\n", this_parameter)
        print("Model Log-Likelihood Value =", this_llk)
        print("Number of Free Parameters =", this_df)

    work_params = pandas.DataFrame(numpy.zeros(shape=(no_full_param, (no_y_cat - 1))))
    work_params = work_params.set_index(keys=full_x.columns)
    full_params = pandas.merge(work_params, this_parameter, how="left", left_index=True, right_index=True)
    full_params = full_params.drop(columns='0_x').fillna(0.0)

    return (this_llk, this_df, full_params)

In [4]:
# load data file
purchase_likelihood = pandas.read_csv('Purchase_Likelihood.csv', delimiter=',', usecols=['group_size', 'homeowner', 'married_couple', 'A'])
purchase_likelihood = purchase_likelihood.dropna()
no_objs = purchase_likelihood.shape[0]
y = purchase_likelihood['A'].astype('category')
# create dummy variables
x_gs = pandas.get_dummies(purchase_likelihood[['group_size']].astype('category'))
x_ho = pandas.get_dummies(purchase_likelihood[['homeowner']].astype('category'))
x_mc = pandas.get_dummies(purchase_likelihood[['married_couple']].astype('category'))

In [23]:
# intercept only model
design_x = pandas.DataFrame(y.where(y.isnull(), 1))
llk0, df0, full_params0 = build_mnlogit(design_x, y, debug='Y')

Column Numbers of the Non-redundant Columns:
(0,)
Optimization terminated successfully.
         Current function value: 0.895013
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665247
Method:                           MLE   Df Model:                            0
Date:                Fri, 01 Nov 2019   Pseudo R-squ.:               6.440e-11
Time:                        11:15:16   Log-Likelihood:            -5.9541e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                       nan
       A=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
A              1.0869      0.003    356.296      0.000    

In [25]:
# intercept + group_size
design_x = stats.add_constant(x_gs, prepend=True)
llk1_gs, df1_gs, full_params1_gs = build_mnlogit (design_x, y, debug = 'Y')
test_dev_gs = 2 * (llk1_gs - llk0)
test_df_gs = df1_gs - df0
test_p_value_gs = scipy.stats.chi2.sf(test_dev_gs, test_df_gs)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3)
Optimization terminated successfully.
         Current function value: 0.894271
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665241
Method:                           MLE   Df Model:                            6
Date:                Fri, 01 Nov 2019   Pseudo R-squ.:               0.0008293
Time:                        11:30:10   Log-Likelihood:            -5.9491e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                4.348e-210
         A=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.5446      0.085      6.441 

In [26]:
# intercept + group_size + homeowner
design_x = x_gs
design_x = design_x.join(x_ho)
design_x = stats.add_constant(design_x, prepend=True)
llk2_gs_ho, df2_gs_ho, full_params2_gs_ho = build_mnlogit (design_x, y, debug = 'Y')
test_dev_gs_ho = 2 * (llk2_gs_ho - llk1_gs)
test_df_gs_ho = df2_gs_ho - df1_gs
test_p_value_gs_ho = scipy.stats.chi2.sf(test_dev_gs_ho, test_df_gs_ho)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5)
Optimization terminated successfully.
         Current function value: 0.889861
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665239
Method:                           MLE   Df Model:                            8
Date:                Fri, 01 Nov 2019   Pseudo R-squ.:                0.005757
Time:                        11:37:40   Log-Likelihood:            -5.9198e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
         A=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.6231      0.085      7.3

In [27]:
# intercept + group_size + homeowner + married_couple
design_x = x_gs
design_x = design_x.join(x_ho)
design_x = design_x.join(x_mc)
design_x = stats.add_constant(design_x, prepend=True)
llk3_gs_ho_mc, df3_gs_ho_mc, full_params3_gs_ho_mc = build_mnlogit (design_x, y, debug = 'Y')
test_dev_gs_ho_mc = 2 * (llk3_gs_ho_mc - llk2_gs_ho)
test_df_gs_ho_mc = df3_gs_ho_mc - df2_gs_ho
test_p_value_gs_ho_mc = scipy.stats.chi2.sf(test_dev_gs_ho_mc, test_df_gs_ho_mc)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5, 7)
Optimization terminated successfully.
         Current function value: 0.889797
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665237
Method:                           MLE   Df Model:                           10
Date:                Fri, 01 Nov 2019   Pseudo R-squ.:                0.005828
Time:                        11:46:09   Log-Likelihood:            -5.9194e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
             A=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.6282     

In [28]:
# intercept + group_size + homeowner + married_couple + group_size * homeowner
design_x = x_gs
design_x = design_x.join(x_ho)
design_x = design_x.join(x_mc)
# create the columns for the group_size * homeowner interaction effect
x_gsho = create_interaction(x_gs, x_ho)
design_x = design_x.join(x_gsho)
design_x = stats.add_constant(design_x, prepend=True)
llk4_gs_ho_mc_gsho, df4_gs_ho_mc_gsho, full_params4_gs_ho_mc_gsho = build_mnlogit(design_x, y, debug='Y')
test_dev_gs_ho_mc_gsho = 2 * (llk4_gs_ho_mc_gsho - llk3_gs_ho_mc)
test_df_gs_ho_mc_gsho = df4_gs_ho_mc_gsho - df3_gs_ho_mc
test_p_value_gs_ho_mc_gsho = scipy.stats.chi2.sf(test_dev_gs_ho_mc_gsho, test_df_gs_ho_mc_gsho)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5, 7, 9, 11, 13)
Optimization terminated successfully.
         Current function value: 0.889606
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665231
Method:                           MLE   Df Model:                           16
Date:                Fri, 01 Nov 2019   Pseudo R-squ.:                0.006041
Time:                        12:00:55   Log-Likelihood:            -5.9181e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                       A=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
c

In [29]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section a)")
print("=" * 50)
# a)	(5 points) List the aliased parameters that you found in your model.
# intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple
design_x = x_gs
design_x = design_x.join(x_ho)
design_x = design_x.join(x_mc)
# create the columns for the group_size * homeowner interaction effect
x_gsho = create_interaction(x_gs, x_ho)
design_x = design_x.join(x_gsho)
design_x = stats.add_constant(design_x, prepend=True)
# create the columns for the homeowner * married_couple interaction effect
x_homc = create_interaction(x_ho, x_mc)
design_x = design_x.join(x_homc)
design_x = stats.add_constant(design_x, prepend=True)
llk5_gs_ho_mc_gsho_homc, df5_gs_ho_mc_gsho_homc, full_params5_gs_ho_mc_gsho_homc = build_mnlogit(design_x, y, debug='Y')
test_dev_gs_ho_mc_gsho_homc = 2 * (llk5_gs_ho_mc_gsho_homc - llk4_gs_ho_mc_gsho)
test_df_gs_ho_mc_gsho_homc = df5_gs_ho_mc_gsho_homc - df4_gs_ho_mc_gsho
test_p_value_gs_ho_mc_gsho_homc = scipy.stats.chi2.sf(test_dev_gs_ho_mc_gsho_homc, test_df_gs_ho_mc_gsho_homc)


ML-Assignment 4-Question 1-Section a)
Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5, 7, 9, 11, 13, 17)
Optimization terminated successfully.
         Current function value: 0.889553
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665229
Method:                           MLE   Df Model:                           18
Date:                Fri, 01 Nov 2019   Pseudo R-squ.:                0.006101
Time:                        12:20:21   Log-Likelihood:            -5.9177e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           A=1       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------

In [30]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section b)")
print("=" * 50)
# b)	(5 points) How many degrees of freedom do you have in your model?
print(f'Degree of Freedom = {df5_gs_ho_mc_gsho_homc}')


ML-Assignment 4-Question 1-Section b)
Degree of Freedom = 20


In [31]:
# define a function that performs the Chi-square test
def chi_square_test (x_cat, y_cat, debug = 'N'):

    obs_count = pandas.crosstab(index = x_cat, columns = y_cat, margins = False, dropna = True)
    c_total = obs_count.sum(axis = 1)
    r_total = obs_count.sum(axis = 0)
    n_total = numpy.sum(r_total)
    exp_count = numpy.outer(c_total, (r_total / n_total))

    if (debug == 'Y'):
        print('Observed Count:\n', obs_count)
        print('Column Total:\n', c_total)
        print('Row Total:\n', r_total)
        print('Overall Total:\n', n_total)
        print('Expected Count:\n', exp_count)
        print('\n')
       
    chi_sq_stat = ((obs_count - exp_count)**2 / exp_count).to_numpy().sum()
    chi_sq_df = (obs_count.shape[0] - 1.0) * (obs_count.shape[1] - 1.0)
    chi_sq_sig = scipy.stats.chi2.sf(chi_sq_stat, chi_sq_df)

    cramer_v = chi_sq_stat / n_total
    if (c_total.size > r_total.size):
        cramer_v = cramer_v / (r_total.size - 1.0)
    else:
        cramer_v = cramer_v / (c_total.size - 1.0)
    cramer_v = numpy.sqrt(cramer_v)

    return(chi_sq_stat, chi_sq_df, chi_sq_sig, cramer_v)

In [32]:
# define a function that performs the deviance test
def deviance_test (x_int, y_cat, debug = 'N' ):

    y = y_cat.astype('category')

    x = numpy.where(y_cat.notnull(), 1, 0)
    obj_logit = stats.MNLogit(y, x)
    this_fit = obj_logit.fit(method = 'newton', full_output = True, maxiter = 100, tol = 1e-8)
    this_parameter = this_fit.params
    llk0 = obj_logit.loglike(this_parameter.values)

    if (debug == 'Y'):
        print(this_fit.summary())
        print("Model Log-Likelihood Value =", llk0)
        print('\n')

    x = stats.add_constant(x_int, prepend = True)
    obj_logit = stats.MNLogit(y, x)
    this_fit = obj_logit.fit(method = 'newton', full_output = True, maxiter = 100, tol = 1e-8)
    this_parameter = this_fit.params
    llk1 = obj_logit.loglike(this_parameter.values)

    if (debug == 'Y'):
        print(this_fit.summary())
        print("Model Log-Likelihood Value =", llk1)

    deviance_stat = 2.0 * (llk1 - llk0)
    deviance_df = (len(y.cat.categories) - 1.0)
    deviance_sig = scipy.stats.chi2.sf(deviance_stat, deviance_df)

    mc_fadden_r_sq = 1.0 - (llk1 / llk0)

    return(deviance_stat, deviance_df, deviance_sig, mc_fadden_r_sq)

In [33]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section c)")
print("=" * 50)
# c)	(10 points) After entering a model effect, calculate the Deviance test statistic, its degrees of freedom, and its significance value between the current model and the previous model.  List your Deviance test results by the model effects in a table.
print('Deviance Chi=Square Test')
print(f'==>for (Intercept + group_size) model')
print('Chi-Square Statistic = ', test_dev_gs)
print('  Degrees of Freedom = ', test_df_gs)
print('        Significance = ', test_p_value_gs)
print(f'==>for (Intercept + group_size + homeowner) model')
print('Chi-Square Statistic = ', test_dev_gs_ho)
print('  Degrees of Freedom = ', test_df_gs_ho)
print('        Significance = ', test_p_value_gs_ho)
print(f'==>for (Intercept + group_size + homeowner + married_couple) model')
print('Chi-Square Statistic = ', test_dev_gs_ho_mc)
print('  Degrees of Freedom = ', test_df_gs_ho_mc)
print('        Significance = ', test_p_value_gs_ho_mc)
print(f'==>for (Intercept + group_size + homeowner + married_couple + group_size * homeowner) model')
print('Chi-Square Statistic = ', test_dev_gs_ho_mc_gsho)
print('  Degrees of Freedom = ', test_df_gs_ho_mc_gsho)
print('        Significance = ', test_p_value_gs_ho_mc_gsho)
print(f'==>for (Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple) model')
print('Chi-Square Statistic = ', test_dev_gs_ho_mc_gsho_homc)
print('  Degrees of Freedom = ', test_df_gs_ho_mc_gsho_homc)
print('        Significance = ', test_p_value_gs_ho_mc_gsho_homc)


ML-Assignment 4-Question 1-Section c)
Deviance Chi=Square Test
==>for (Intercept + group_size) model
Chi-Square Statistic =  987.5766005262267
  Degrees of Freedom =  6
        Significance =  4.347870389027117e-210
==>for (Intercept + group_size + homeowner) model
Chi-Square Statistic =  5867.781500353245
  Degrees of Freedom =  2
        Significance =  0.0
==>for (Intercept + group_size + homeowner + married_couple) model
Chi-Square Statistic =  84.5780023841653
  Degrees of Freedom =  2
        Significance =  4.306457217534288e-19
==>for (Intercept + group_size + homeowner + married_couple + group_size * homeowner) model
Chi-Square Statistic =  254.0781253632158
  Degrees of Freedom =  6
        Significance =  5.512105969198056e-52
==>for (Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple) model
Chi-Square Statistic =  70.84227677015588
  Degrees of Freedom =  2
        Significance =  4.13804354648637e-16


In [35]:
# apply chi-square and deviance test on given data
predictor = ['group_size', 'homeowner', 'married_couple']

test_result = pandas.DataFrame(index = predictor, columns = ['Test', 'Statistic', 'DF', 'Significance', 'Association', 'Measure'])
    
for pred in predictor:
    deviance_stat, deviance_df, deviance_sig, mc_fadden_r_sq = deviance_test(purchase_likelihood[pred], purchase_likelihood['A'], debug = 'N')
    test_result.loc[pred] = ['Deviance', deviance_stat, deviance_df, deviance_sig, 'McFadden''s R^2', mc_fadden_r_sq]

print(test_result[test_result['Test'] == 'Deviance'])

Optimization terminated successfully.
         Current function value: 0.895013
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.894766
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.895013
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.890305
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.895013
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.894477
         Iterations 5
                    Test Statistic DF Significance    Association      Measure
group_size      Deviance    329.43  2  2.91862e-72  McFaddens R^2  0.000276643
homeowner       Deviance   6264.24  2            0  McFaddens R^2   0.00526047
married_couple  Deviance   714.265  2    7.93e-156  McFaddens R^2  0.000599813


In [43]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section d)")
print("=" * 50)
# d)	(5 points) Calculate the Feature Importance Index as the negative base-10 logarithm of the significance value.  List your indices by the model effects.
print(f'Feature Importance Index for (Intercept + group_size) \n= {-(numpy.log10(test_p_value_gs))}')
print(f'Feature Importance Index for (Intercept + group_size + homeowner) \n= {-(numpy.log10(test_p_value_gs_ho))}')
print(f'Feature Importance Index for (Intercept + group_size + homeowner + married_couple) \n= {-(numpy.log10(test_p_value_gs_ho_mc))}')
print(f'Feature Importance Index for (Intercept + group_size + homeowner + married_couple + group_size * homeowner) \n= {-(numpy.log10(test_p_value_gs_ho_mc_gsho))}')
print(f'Feature Importance Index for (Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple) \n= {-(numpy.log10(test_p_value_gs_ho_mc_gsho_homc))}')


ML-Assignment 4-Question 1-Section d)
Feature Importance Index for (Intercept + group_size) 
= 209.36172341080683
Feature Importance Index for (Intercept + group_size + homeowner) 
= inf
Feature Importance Index for (Intercept + group_size + homeowner + married_couple) 
= 18.36587986292153
Feature Importance Index for (Intercept + group_size + homeowner + married_couple + group_size * homeowner) 
= 51.25868244179064
Feature Importance Index for (Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple) 
= 15.38320494337081


C:\Users\yashm\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys


In [8]:
# create train data for train MNLogit model
design_x = x_gs
design_x = design_x.join(x_ho)
design_x = design_x.join(x_mc)
# Create the columns for the group_size * homeowner interaction effect
x_gsho = create_interaction(x_gs, x_ho)
design_x = design_x.join(x_gsho)
design_x = stats.add_constant(design_x, prepend=True)
# Create the columns for the homeowner * married_couple interaction effect
x_homc = create_interaction(x_ho, x_mc)
design_x = design_x.join(x_homc)
design_x = stats.add_constant(design_x, prepend=True)

# train MNLogit model
logit = stats.MNLogit(y, design_x)
this_fit = logit.fit(method='newton', full_output=True, maxiter=100, tol=1e-8)

C:\Users\yashm\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


         Current function value: 0.889553
         Iterations: 100


C:\Users\yashm\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [20]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section e)")
print("=" * 50)
# e)	(10 points) For each of the sixteen possible value combinations of the three features, calculate the predicted probabilities for A = 0, 1, 2 based on the multinomial logistic model.  List your answers in a table with proper labelling.
gs_d = [1,2,3,4]
ho_d = [0,1]
mc_d = [0,1]
A_d = [0,1,2]
# create all possible combinations of features
x_data = []

for gsd in gs_d:
    for hod in ho_d:
        for mcd in mc_d:
            data = [gsd,hod,mcd]
            x_data = x_data + [data]

# create input data for prediction through the model
x_input = pandas.DataFrame(x_data, columns=['group_size','homeowner','married_couple'])
x_gs = pandas.get_dummies(x_input[['group_size']].astype('category'))
x_ho = pandas.get_dummies(x_input[['homeowner']].astype('category'))
x_mc = pandas.get_dummies(x_input[['married_couple']].astype('category'))
x_design = x_gs
x_design = x_design.join(x_ho)
x_design = x_design.join(x_mc)
# Create the columns for the group_size * homeowner interaction effect
x_gsho = create_interaction(x_gs, x_ho)
x_design = x_design.join(x_gsho)
x_design = stats.add_constant(x_design, prepend=True)
# Create the columns for the homeowner * married_couple interaction effect
x_homc = create_interaction(x_ho, x_mc)
x_design = x_design.join(x_homc)
x_design = stats.add_constant(x_design, prepend=True)
A_pred = this_fit.predict(exog = x_design)
# map the columns
A_pred['p_a_0'] = A_pred[0]
A_pred['p_a_1'] = A_pred[1]
A_pred['p_a_2'] = A_pred[2]

A_output = pandas.concat([x_input, A_pred[['p_a_0','p_a_1','p_a_2']]],axis=1)
print(A_output)


ML-Assignment 4-Question 1-Section e)
    group_size  homeowner  married_couple     p_a_0     p_a_1     p_a_2
0            1          0               0  0.259651  0.589175  0.151174
1            1          0               1  0.260092  0.592106  0.147802
2            1          1               0  0.183602  0.682030  0.134368
3            1          1               1  0.154023  0.709918  0.136059
4            2          0               0  0.221936  0.621105  0.156959
5            2          0               1  0.222321  0.624216  0.153463
6            2          1               0  0.202510  0.659773  0.137718
7            2          1               1  0.170552  0.689450  0.139999
8            3          0               0  0.239570  0.604616  0.155814
9            3          0               1  0.239992  0.607660  0.152348
10           3          1               0  0.301140  0.531297  0.167563
11           3          1               1  0.259017  0.567017  0.173966
12           4          0

In [67]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section f)")
print("=" * 50)
# f)	(5 points) Based on your model, what values of group_size, homeowner, and married_couple will maximize the odds value Prob(A=1) / Prob(A = 0)?  What is that maximum odd value?
A_output['odd value(p_a_1/p_a_0)'] = A_output['p_a_1'] / A_output['p_a_0']
print(A_output[['group_size','homeowner','married_couple','odd value(p_a_1/p_a_0)']])
print(A_output.loc[A_output['odd value(p_a_1/p_a_0)'].idxmax()])


ML-Assignment 4-Question 1-Section f)
    group_size  homeowner  married_couple  odd value(p_a_1/p_a_0)
0            1          0               0                2.269108
1            1          0               1                2.276527
2            1          1               0                3.714708
3            1          1               1                4.609169
4            2          0               0                2.798576
5            2          0               1                2.807726
6            2          1               0                3.257983
7            2          1               1                4.042470
8            3          0               0                2.523754
9            3          0               1                2.532006
10           3          1               0                1.764286
11           3          1               1                2.189107
12           4          0               0                3.443386
13           4          0            

In [21]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section g)")
print("=" * 50)
# g)	(5 points) Based on your model, what is the odds ratio for group_size = 3 versus group_size = 1, and A = 2 versus A = 0?  Mathematically, the odds ratio is (Prob(A=2)/Prob(A=0) | group_size = 3) / ((Prob(A=2)/Prob(A=0) | group_size = 1).
# calculate odd for (Prob(A=2)/Prob(A=0) | group_size = 3)
pr_a_2_g_gs_3 = (purchase_likelihood[purchase_likelihood['group_size']==3].groupby('A').size()[2]/purchase_likelihood[purchase_likelihood['group_size']==3].shape[0]) 
pr_a_0_g_gs_3 = (purchase_likelihood[purchase_likelihood['group_size']==3].groupby('A').size()[0]/purchase_likelihood[purchase_likelihood['group_size']==3].shape[0])
o1 = pr_a_2_g_gs_3/pr_a_0_g_gs_3
# calculate odd for (Prob(A=2)/Prob(A=0) | group_size = 1)
pr_a_2_g_gs_1 = (purchase_likelihood[purchase_likelihood['group_size']==1].groupby('A').size()[2]/purchase_likelihood[purchase_likelihood['group_size']==1].shape[0]) 
pr_a_0_g_gs_1 = (purchase_likelihood[purchase_likelihood['group_size']==1].groupby('A').size()[0]/purchase_likelihood[purchase_likelihood['group_size']==1].shape[0])
o2 = pr_a_2_g_gs_1/pr_a_0_g_gs_1
# calculate odd ratio
o_r = o1/o2
print(o_r)


ML-Assignment 4-Question 1-Section g)
1.0249543364157785


In [57]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 1-Section h)")
print("=" * 50)
# h)	(5 points) Based on your model, what is the odds ratio for homeowner = 1 versus homeowner = 0, and A = 0 versus A = 1?  Mathematically, the odds ratio is (Prob(A=0)/Prob(A=1) | homeowner = 1) / ((Prob(A=0)/Prob(A=1) | homeowner = 0).
# calculate odd for (Prob(A=0)/Prob(A=1) | homeowner = 1)
pr_a_0_g_ho_1 = (purchase_likelihood[purchase_likelihood['homeowner']==1].groupby('A').size()[0]/purchase_likelihood[purchase_likelihood['homeowner']==1].shape[0]) 
pr_a_1_g_ho_1 = (purchase_likelihood[purchase_likelihood['homeowner']==1].groupby('A').size()[1]/purchase_likelihood[purchase_likelihood['homeowner']==1].shape[0])
o1 = pr_a_0_g_ho_1/pr_a_1_g_ho_1
# calculate odd for (Prob(A=0)/Prob(A=1) | homeowner = 0)
pr_a_0_g_ho_0 = (purchase_likelihood[purchase_likelihood['homeowner']==0].groupby('A').size()[0]/purchase_likelihood[purchase_likelihood['homeowner']==0].shape[0]) 
pr_a_1_g_ho_0 = (purchase_likelihood[purchase_likelihood['homeowner']==0].groupby('A').size()[1]/purchase_likelihood[purchase_likelihood['homeowner']==0].shape[0])
o2 = pr_a_0_g_ho_0/pr_a_1_g_ho_0
# calculate odd ratio
o_r = o1/o2
print(o_r)


ML-Assignment 4-Question 1-Section h)
0.6232245044401726
